# Catálogo de la SSPM

## Setup

In [16]:
import pandas as pd

In [17]:
df = pd.read_excel("/Users/abenassi/github/series-tiempo/catalogo/datos/catalogo-sspm.xlsx", "Parametros ETL")

In [44]:
series = df.distribution_iedFileURL.unique()

In [45]:
with open("/Users/abenassi/github/series-tiempo/catalogo/datos/excels_urls.txt", "wb") as f:
    for url in series:
        f.write(url)

In [24]:
series

array([u'http://www.economia.gob.ar/download/infoeco/actividad_ied.xlsx'], dtype=object)

### Importación de metodos y clases

In [1]:
import arrow
import os, sys
sys.path.insert(0, os.path.abspath(".."))
from openpyxl import load_workbook
from pydatajson import DataJson #lib y clase
from pydatajson.readers import read_catalog # lib, modulo ... metodo Lle el catalogo -json o xlsx o (local o url) dicc- y lo transforma en un diccionario de python
from pydatajson.writers import write_json_catalog

### Declaración de variables y paths

In [2]:
#completar con lo que corresponda
ORGANISMO = 'sspm'
catalogo_xlsx = "catalogo_sspm.xlsx"

In [3]:
#NO MODIFICAR

#Creo la estructura de directorios necesaria si no existe
if not os.path.isdir("archivos-generados"):
    os.mkdir("archivos-generados")

for directorio in ["jsons", "reportes", "configuracion"]:
    path = os.path.join("archivos-generados", directorio)
    if not os.path.isdir(path):
        os.mkdir(path)

# Declaro algunas variables de interés
HOY = arrow.now().format('YYYY-MM-DD-HH_mm')
catalogo_a_json = os.path.join("archivos-generados","jsons","catalogo-{}-{}.json".format(ORGANISMO, HOY))
reporte_datasets = os.path.join("archivos-generados", "reportes", "reporte-catalogo-{}-{}.xlsx".format(ORGANISMO, HOY))
archivo_config_sin_reporte = os.path.join("archivos-generados", "configuracion", "archivo-config_-{}-{}-sinr.csv".format(ORGANISMO, HOY))
archivo_config_con_reporte = os.path.join("archivos-generados", "configuracion", "archivo-config-{}-{}-conr.csv".format(ORGANISMO, HOY))

## Validación del archivo xlsx y transformación a json

### Validación del catálogo en xlsx

In [4]:
catalogo = read_catalog(catalogo_xlsx, default_values={
    "dataset_issued": arrow.now().isoformat(),
    "distribution_issued": arrow.now().isoformat()
})
# En el caso que quiera trabajarse con un archivo remoto:
#catalogo = read_catalog("https://raw.githubusercontent.com/datosgobar/pydatajson/master/tests/samples/catalogo_justicia.json")

### Transformación del catálogo, de xlsx a json

In [5]:
write_json_catalog(catalogo, catalogo_a_json)

##write_json_catalog(catalog, target_file)  escrie un dicc a un archivo json

## Validación del catalogo en json y harvesting

### Validación del catálogo en json

Instanciación de la clase DataJson

In [16]:
dj = DataJson("catalog.json", "/Users/abenassi/Dropbox/ssipyga_pers/15. Series de Tiempo/schemas")


Validación -V/F- del catálogo en json

In [17]:

dj.is_valid_catalog(catalogo)


False

Validación detallada del catálogo en json

In [19]:
validation_result = dj.validate_catalog(catalogo)

In [24]:
filter(lambda x: x["status"] == "ERROR", validation_result["error"]["dataset"])

[{u'errors': [{u'error_code': 2,
    u'instance': 30L,
    u'message': '30L is not valid under any of the given schemas',
    u'path': [u'dataset',
     14,
     u'distribution',
     0,
     u'field',
     17,
     u'description'],
    u'validator': u'anyOf',
    u'validator_value': [{u'type': u'string'}, {u'type': u'null'}]},
   {u'error_code': 2,
    u'instance': 30L,
    u'message': '30L is not valid under any of the given schemas',
    u'path': [u'dataset',
     14,
     u'distribution',
     1,
     u'field',
     17,
     u'description'],
    u'validator': u'anyOf',
    u'validator_value': [{u'type': u'string'}, {u'type': u'null'}]}],
  u'status': u'ERROR',
  u'title': u'Encuesta industrial. \xcdndice de Volumen F\xedsico (IVF).  (Base 1997)'},
 {u'errors': [{u'error_code': 2,
    u'instance': 30L,
    u'message': '30L is not valid under any of the given schemas',
    u'path': [u'dataset',
     15,
     u'distribution',
     0,
     u'field',
     17,
     u'description'],
    u

### Harvesting

Generación del archivo de reporte de datasets

In [9]:
dj.generate_datasets_report(catalogo, harvest='valid',export_path=reporte_datasets)
# proceso el repote, 0 y 1s

Generación del archivo de configuración para el harvester

In [10]:
#  usando el reporte
dj.generate_harvester_config(harvest='report', report=reporte_datasets, export_path=archivo_config_con_reporte)

In [11]:
# sin usar el reporte
dj.generate_harvester_config(catalogs=catalogo, harvest='valid', export_path=archivo_config_sin_reporte)
#(catalogs=None, harvest=u'valid', report=None, export_path=None)